# 1. Common import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 7)

# 2. Frame size

In [ ]:
FRAME_WIDTH = 1920
FRAME_HEIGHT = 1080

# 3. Filename from index

In [ ]:
ZERO_PADDING = 6

def get_filename(index):
    return "src/Csv/{}.txt".format(str(index).zfill(ZERO_PADDING))

# 4. Pedestrian info

In [ ]:
NUMBER_OF_PEDESTRIANS = 12273

def download_pedestrian(index):
    error_message_template = "pedestrian_separation_to_test_and_train_according_to_start_point.ipynbestrian number should be between 0 and {max}; given number: {id}"
    assert(0 <= index < NUMBER_OF_PEDESTRIANS), error_message_template.format(max=NUMBER_OF_PEDESTRIANS-1, id=index)
    filename = get_filename(index)
    data = pd.read_csv(filename, index_col=0)
    return data

# 5. Create and save plot.bar

In [ ]:
ind = np.arange(25)
width = 0.4
test_bar = plt.bar(ind - width/2, test_by_labels_normalized, width, color='r')
train_bar = plt.bar(ind + width/2, train_by_labels_normalized, width, color='b')

plt.ylabel('Percent')
plt.xlabel('Label number')
plt.title('Pedestrian percent by group and label')
plt.xticks(ind + width/2., range(1, 26))
plt.legend((test_bar[0], train_bar[0]), ('Test', 'Train'))
plt.savefig('src/Plots/161002_pedestrian_separation_percent_by_label.png')

# 6. Distances between arrays

In [ ]:
# array1: [sX1; sY1; sX2; sY2; ...]
# array2: [eX1; eY1; eX2; eY2; ...]
# output: [dist((sX1, sY1), (eX1, eY1)), dist((sX1, sY1), (eX1, eY1)),
#          dist((sX2, sY2), (eX2, eY2)), dist((sX2, sY2), (eX2, eY2)), ...]

def distance_for_each_point(array1, array2):
    error_template = "Array lengths should be equal. len(array1): {len1}, len(array2): {len2}"
    assert (len(array1) == len(array2)), error_template.format(len1=len(array1), len2=len(array2))
    
    length = len(array1)
    distance = np.array([np.linalg.norm(array1[i:i+2] - array2[i:i+2]) for i in range(0,length,2)])
    distance = np.array([[d, d] for d in distance]).flatten()